# Example using an heterogeneous pool

The library also support a heterogenous pool of classifiers. A pool is called heterogeneous when different classifier models are used to generate a diverse pool of classifiers (e.g., svm, decision tree, naive bayes...)

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Example of DCS techniques
from DCS.OLA import OLA
from DCS.APriori import APriori
from DCS.MCB import MCB
# Example of DES techniques
from DES.KNORAE import KNORAE
from DES.DESP import DESP
from DES.KNORAU import KNORAU

ModuleNotFoundError: No module named 'DCS'

## Loading a classification dataset and preparing the data

In [ ]:
data = load_breast_cancer()
X = data.data
y = data.target

# split the data into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# Split the data into training and DSEL for DS techniques
X_train, X_dsel, y_train, y_dsel = train_test_split(X_train, y_train, test_size=0.5)

## Training an heterogeneous pool of classifiers

In this example we train a pool composed of:
1 - Perceptron classifier
2 - Linear SVM
3 - Gaussian SVM
4 - Naive Bayes
5 - Decision tree
6 - 1NN

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.trees import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV

model_perceptron = CalibratedClassifierCV(Perceptron(), cv=3, method='sigmoid').fit(X_train, y_train)
model_linear_svm = LinearSVC(probability=True).fit(X_train, y_train)
model_svc = SVC(probability=True).fit(X_train, y_train)
model_bayes = GaussianNB().fit(X_train, y_train)
model_bayes2 = MultinomialNB().fit(X_train, y_train)
model_tree = DecisionTreeClassifier().fit(X_train, y_train)
model_knn = KNeighborsClassifier(k=1).fit(X_train, y_train)
pool_classifiers = [model_perceptron, model_linear_svm, model_svc, model_bayes, model_bayes2, model_tree, model_knn]

## Initializing DS techniques

Here we initialize the DS techniques. Three DCS and three DES techniques are considered in this example. In this example, we specify the size of the region of competence (k = 5)

In [ ]:
# DES techniques
knorau = KNORAU(pool_classifiers, k=5)
kne = KNORAE(pool_classifiers, k=5)
desp = DESP(pool_classifiers, k=5)
# DCS techniques
ola = OLA(pool_classifiers, k=5)
mcb = MCB(pool_classifiers, k=5)
apriori = APriori(pool_classifiers, k=5)

## Fitting the DS techniques

The function fit(data, target) is used to fit each dynamic selection method. The fit function prepares the algorithm that estimates the region of competence (e.g., K-NN algorithm) and pre-process information required to apply the DS techniques.

In [ ]:
knorau.fit(X_dsel, y_dsel)
kne.fit(X_dsel, y_dsel)
desp.fit(X_dsel, y_dsel)
ola.fit(X_dsel, y_dsel)
mcb.fit(X_dsel, y_dsel)
apriori.fit(X_dsel, y_dsel)

## Calculate classification accuracy of each technique

In this case, the first result is the classification accuracy of the random forest classifier, which combines the outputs of each base decision tree using the majority voting scheme. 

Using DS techniques, instead of combining all decision trees, only the ones that are more competent locally are used for classification. In the case of DCS techniques, the decision tree that is most competent locally is used for prediction. In the case of DES techniques, an ensemble containing the most competent decision trees are selected to predict the label of a given query sample.

In [ ]:
print('Classification accuracy of Majority voting the pool: ', pool_classifiers.score(X_test, y_test))
print('Classification accuracy of KNORA-Union: ', knorau.score(X_test, y_test))
print('Classification accuracy of KNORA-Eliminate: ', kne.score(X_test, y_test))
print('Classification accuracy of DESP: ', desp.score(X_test, y_test))
print('Classification accuracy of OLA: ', ola.score(X_test, y_test))
print('Classification accuracy of A priori: ', apriori.score(X_test, y_test))